In [1]:
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

from script import *
from feature_engineering import *
import db_operations as dbop
from constants import *
import data_process as dp

import time

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
start = 20100101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt']/=1e5
df_d = df_d.astype('float32')
print(df_d.columns)
print(df_d.shape)
df_d.info(memory_usage='deep')

select * from stock_day where date>=20100101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(3069141, 11)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3069141 entries, (000001.SZ, 2010-01-04 00:00:00) to (603997.SH, 2020-04-14 00:00:00)
Data columns (total 11 columns):
open          float32
high          float32
low           float32
close         float32
vol           float32
amt           float32
adj_factor    float32
avg           float32
close0        float32
avg0          float32
pct           float32
dtypes: float32(11)
memory usage: 220.6 MB


In [4]:
df_d_basic = dbop.create_df(cursor, STOCK_DAILY_BASIC[TABLE], start=start)
df_d_basic = dp.prepare_stock_d_basic(df_d_basic)
#
df_d_basic["pb*pe_ttm"] = df_d_basic["pb"] * df_d_basic["pe_ttm"]
df_d_basic["pb*pe"] = df_d_basic["pb"] * df_d_basic["pe"]

df_d_basic = df_d_basic.astype('float32')
print(df_d_basic.columns)
print(df_d_basic.shape)
df_d_basic.info(memory_usage='deep')

select * from stock_daily_basic where date>=20100101
Index(['close', 'turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe',
       'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share', 'float_share',
       'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm', 'pb*pe'],
      dtype='object')
(2909103, 16)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2909103 entries, (000001.SZ, 2010-01-04 00:00:00) to (603997.SH, 2020-04-14 00:00:00)
Data columns (total 16 columns):
close              float32
turnover_rate      float32
turnover_rate_f    float32
volume_ratio       float32
pe                 float32
pe_ttm             float32
pb                 float32
ps                 float32
ps_ttm             float32
total_share        float32
float_share        float32
free_share         float32
total_mv           float32
circ_mv            float32
pb*pe_ttm          float32
pb*pe              float32
dtypes: float32(16)
memory usage: 188.8 MB


In [5]:
df = df_d
periods = [5,10,20,30,60,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]
print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma', 'close/5ma', 'close/10ma', 'close/20ma', 'close/30ma',
       'close/60ma', 'close/120ma', 'close/250ma'],
      dtype='object')


In [6]:
df = df.join(df_d_basic.drop(columns='close'),how='inner')

In [7]:
index = df_d.index[(df_d['close/120ma']<=0.69)
                 & (df_d['close/30ma']<=0.778)
                 & (df_d['close/60ma']<=0.65)
                 & (df_d['close/60ma']>=0.55)
                ]
# print(df['turnover_rate_f'].head())
# df['p10mean_turnover_rate_f'] = df.reset_index('code').groupby('code')['turnover_rate_f'].rolling(10).mean()
df['p10mean_turnover_rate'] = df.reset_index('code').groupby('code')['turnover_rate'].rolling(10).mean()

stock_mask = (df['circ_mv']>50e4) & (df['total_mv']>100e4) & (df['close0']>3.5) & (df['p10mean_turnover_rate']<5) & (df['amt']>0.5)
print(stock_mask.sum(),stock_mask.mean())
# print(len(index))

1053212 0.36227598359801444


In [8]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [9]:
features = []
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    features.append('{}/{}'.format(col1,col2))
print(features)
# result_50 = ml.assess_feature3(df.loc[stock_mask,features],df_r.r,q_bin=50)
# result_100 = ml.assess_feature3(df.loc[stock_mask,features],df_r.r,q_bin=100)

['close/5ma', 'close/10ma', 'close/20ma', 'close/30ma', 'close/60ma', 'close/120ma', 'close/250ma']


In [43]:
print(result.sort_values(['median_std','mean_std'],ascending=False))
print(result_50.sort_values(['median_std','mean_std'],ascending=False))
print(result_100.sort_values(['median_std','mean_std'],ascending=False))


             mean_std  median_std  mean_q96%  median_q96%  mean_q4%  \
close/60ma   0.002214    0.002930   0.006737    -0.002907  0.000177   
close/30ma   0.001786    0.002642   0.005013    -0.004687 -0.000075   
close/20ma   0.001518    0.002552   0.004747    -0.005438  0.000216   
close/120ma  0.002525    0.002524   0.006875    -0.003664  0.000394   
close/10ma   0.001263    0.002277   0.004025    -0.005446  0.000436   
close/5ma    0.001634    0.002189   0.005813    -0.005129  0.000456   
close/250ma  0.001411    0.001058   0.005713    -0.006659  0.001670   

             median_q4%  
close/60ma    -0.011214  
close/30ma    -0.011815  
close/20ma    -0.011823  
close/120ma   -0.010571  
close/10ma    -0.011761  
close/5ma     -0.011686  
close/250ma   -0.009766  
             mean_std  median_std  mean_q96%  median_q96%  mean_q4%  \
close/60ma   0.001779    0.002629   0.003023    -0.005295 -0.001769   
close/30ma   0.001455    0.002506   0.001915    -0.006860 -0.001968   
close/120m

In [10]:
# strategy_mask = (
#     (df['close/120ma']<0.7) 
#     & (df['close/60ma']<0.75) & (df['close/60ma']>0.55) 
#     & (df['close/30ma']<0.79) 
#     & (df['close/20ma']<0.82)
#     & (df['close/5ma']<0.95)
#                 )
strategy_mask = (
    (df['close/120ma']<0.72) 
    & (df['close/60ma']<0.77) 
    & (df['close/60ma']>0.55) 
    & (df['close/30ma']<0.82) 
#     & (df['close/20ma']<0.82)
#     & (df['close/5ma']<0.95)
                )
print(strategy_mask.sum())

8286


In [11]:
print(df_r.loc[df.index[stock_mask & strategy_mask],'r'].agg(agg_operations))
# print(df_r2.loc[df.index[stock_mask & strategy_mask],'r'].agg(agg_operations))
print(df_r.loc[df.index[stock_mask & strategy_mask],'r'].reset_index('code')['r'].resample('Q').agg(agg_operations))
df_r.loc[df.index[stock_mask & strategy_mask],:].sort_values('date',ascending=False)[:50]
# df[stock_mask & strategy_mask][['close']].sort_values('date',ascending=False)[:50]

mean         0.030431
median       0.025397
max          0.944298
min         -0.554711
std          0.088527
size      2550.000000
Name: r, dtype: float64
                mean    median       max       min       std  size
date                                                              
2011-12-31 -0.010072 -0.002491  0.081329 -0.103537  0.057601    20
2012-03-31  0.124699  0.122756  0.149573  0.098592  0.016905     6
2012-06-30       NaN       NaN       NaN       NaN       NaN     0
2012-09-30       NaN       NaN       NaN       NaN       NaN     0
2012-12-31  0.049252  0.053061  0.076647  0.019380  0.020764     5
2013-03-31       NaN       NaN       NaN       NaN       NaN     0
2013-06-30  0.000017  0.000000  0.094919 -0.057143  0.030119    59
2013-09-30  0.023552  0.027842  0.109537 -0.066289  0.051436    35
2013-12-31  0.053341  0.053341  0.055400  0.051282  0.002912     2
2014-03-31  0.051143  0.062480  0.074101  0.005512  0.031619     4
2014-06-30  0.035240  0.035240  0.078145

,,buy_at,max,max_idx,idx,is_selled,sell_at,sell_date,r
code,date,,,,,,,,
300207.SZ,2020-04-13,91.053711,98.578812,2131.0,2130,False,NaN,NaT,NaN
002411.SZ,2020-04-10,35.994240,36.994080,2093.0,2092,True,36.369179,NaT,0.010417
002583.SZ,2020-04-03,33.788223,36.149460,2103.0,2100,True,35.137501,2020-04-14,0.039933
002411.SZ,2020-04-03,37.077396,38.910439,2089.0,2088,True,37.077396,2020-04-09,0.000000
002583.SZ,2020-04-02,34.013103,36.149460,2103.0,2099,True,35.137501,2020-04-14,0.033058
002411.SZ,2020-04-02,32.994720,38.910439,2089.0,2087,True,37.493999,2020-04-08,0.136364
300618.SZ,2020-04-01,109.310043,121.387108,752.0,748,True,120.770706,2020-04-09,0.104845
002583.SZ,2020-04-01,32.832481,36.149460,2103.0,2098,True,35.137501,2020-04-14,0.070205
601808.SH,2020-04-01,12.939040,14.795610,2487.0,2485,True,14.613371,2020-04-07,0.129401


In [106]:
df.loc[strategy_mask,['open','close','close0','amt','pct','adj_factor','close/30ma','close/60ma','close/120ma']].sort_values('date',ascending=False)[:50]
# stock_mask.sort_index(level='date').tail(50)
# print(stock_mask.loc[idx['300207.SZ',:],:].tail())
print(df.loc[idx['300207.SZ',:],['circ_mv','total_mv','close0','p10mean_turnover_rate_f','amt','turnover_rate_f','turnover_rate']].tail())

                         circ_mv    total_mv  close0  p10mean_turnover_rate_f  \
code      date                                                                  
300207.SZ 2020-04-07  2178567.25  2466680.75   15.72                  4.87429   
          2020-04-08  2197969.25  2488648.75   15.86                  4.93659   
          2020-04-09  2156393.50  2441574.50   15.56                  4.85817   
          2020-04-10  1981775.50  2243863.50   14.30                  5.13992   
          2020-04-13  1815472.75  2055567.25   13.10                  5.69921   

                            amt  turnover_rate_f  turnover_rate  
code      date                                                   
300207.SZ 2020-04-07   9.109506           6.2898         4.2001  
          2020-04-08   6.978767           4.7744         3.1882  
          2020-04-09   7.080351           4.9246         3.2885  
          2020-04-10   9.848499           7.2678         4.8532  
          2020-04-13  11.169043     

In [56]:
df_r[df_r['r'].isnull()].is_selled.unique()

array([False])

In [12]:
index = df.index[(df['close/120ma']<=0.69)
                 & (df['close/30ma']<=0.778)
                 & (df['close/60ma']<=0.65)
                 & (df['close/60ma']>=0.55)
                ]
print(len(index))

1303


In [17]:

df.loc[index,:].sort_values('date',ascending=False)[:100]
# .sort_values(['code','date'])
df.loc[idx['002506.SZ',:],features].sort_values('date',ascending=False)[:20]
# df_r.loc[index,:].sort_values('date',ascending=False)[:50]

close/5ma  close/10ma  close/20ma  close/30ma  \
code      date                                                        
002506.SZ 2020-04-14   1.013793    0.988568    0.795455    0.692580   
          2020-04-13   0.912509    0.887633    0.699319    0.615775   
          2020-04-10   0.960509    0.933637    0.725664    0.647709   
          2020-04-09   0.990753    0.950269    0.737010    0.662057   
          2020-04-08   0.991464    0.928374    0.723353    0.653397   
          2020-04-07   1.007218    0.905338    0.717122    0.651113   
          2020-04-03   0.964401    0.834033    0.678429    0.620532   
          2020-04-02   0.971519    0.811096    0.679730    0.628583   
          2020-04-01   0.940353    0.775213    0.666451    0.622398   
          2020-03-31   0.875723    0.722805    0.637224    0.600634   
          2020-03-30   0.878415    0.742417    0.672062    0.639418   
          2020-03-27   0.821006    0.719533    0.666000    0.639728   
          2020-03-26   0.839002    0.765095    0.724141    0.701422   
          2020-03-25   0.845070    0.794543    0.758839    0.741944   
          2020-03-24   0.893583    0.863253    0.829172    0.814915   
          2020-03-23   0.958448    0.945658    0.909429    0.899062   
          2020-03-20   0.980769    0.978511    0.941480    0.935952   
          2020-03-19   0.965412    0.967988    0.934941    0.930914   
          2020-03-18   0.975009    0.974087    0.944261    0.941901   
          2020-03-17   1.003795    0.994735    0.966916    0.965975   

                      close/60ma  close/120ma  close/250ma  
code      date                                              
002506.SZ 2020-04-14    0.598128     0.527796     0.487090  
          2020-04-13    0.538451     0.476821     0.441462  
          2020-04-10    0.573312     0.509686     0.473482  
          2020-04-09    0.593824     0.529918     0.494081  
          2020-04-08    0.592447     0.530763     0.496619  
          2020-04-07    0.597160     0.536799     0.504092  
          2020-04-03    0.574458     0.518441     0.488605  
          2020-04-02    0.586251     0.531448     0.502606  
          2020-04-01    0.584784     0.532223     0.505189  
          2020-03-31    0.568374     0.519191     0.494723  
          2020-03-30    0.609571     0.559178     0.534805  
          2020-03-27    0.614694     0.565029     0.542310  
          2020-03-26    0.677800     0.624754     0.601833  
          2020-03-25    0.720808     0.665779     0.643488  
          2020-03-24    0.796452     0.737204     0.714416  
          2020-03-23    0.881266     0.817008     0.793579  
          2020-03-20    0.916085     0.850354     0.827433  
          2020-03-19    0.910394     0.845645     0.824001  
          2020-03-18    0.920410     0.855790     0.835150  
          2020-03-17    0.942931     0.877596     0.857803

In [67]:
df_r[(df_r.is_selled==True) & (df_r.r.isnull())].sort_values('date').head(50)

,,buy_at,max,max_idx,idx,is_selled,sell_at,r
code,date,,,,,,,
000418.SZ,2019-04-30,167.895859,175.670792,2205.0,2203,True,NaN,NaN
002450.SZ,2019-07-03,52.103348,53.277512,1912.0,1910,True,NaN,NaN
600401.SH,2019-07-04,2.830200,3.207560,1392.0,1391,True,NaN,NaN
002477.SZ,2019-10-11,3.269920,3.883030,2016.0,2014,True,NaN,NaN
002143.SZ,2019-11-26,0.895160,0.927130,1984.0,1983,True,NaN,NaN
600240.SH,2020-01-21,3.030120,3.349080,2335.0,2333,True,NaN,NaN
000848.SZ,2020-03-12,145.541504,161.439117,2488.0,2471,True,NaN,NaN
600236.SH,2020-03-12,30.058800,31.209702,2409.0,2395,True,NaN,NaN
600729.SH,2020-03-12,81.136620,87.456543,2317.0,2305,True,NaN,NaN


In [74]:
df_r['curr_date']= df_r.index.get_level_values('date')

In [75]:
df_r.head()

buy_at         max  max_idx  idx  is_selled  \
code      date                                                          
000001.SZ 2010-01-04  852.767456  858.153320      1.0    0       True   
          2010-01-05  834.814453  850.254028      5.0    1       True   
          2010-01-06  822.247375  850.254028      5.0    2       True   
          2010-01-07  807.884949  850.254028      5.0    3       True   
          2010-01-08  843.790955  850.254028      5.0    4       True   

                         sell_date     sell_at         r  curr_date  
code      date                                                       
000001.SZ 2010-01-04  1.262909e+18  807.884949 -0.052632 2010-01-04  
          2010-01-05  1.263254e+18  810.398376 -0.029247 2010-01-05  
          2010-01-06  1.263254e+18  810.398376 -0.014411 2010-01-06  
          2010-01-07  1.263254e+18  810.398376  0.003111 2010-01-07  
          2010-01-08  1.263341e+18  785.264221 -0.069362 2010-01-08

In [78]:
df_r1 = pd.read_parquet(r"database\return_5%_10%_20_8_15% close").loc[idx[:,start_date:],:].sort_index()

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [79]:
df_r2 = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [81]:
print(df_r.r.agg(agg_operations)[:4])
print(df_r1.r.agg(agg_operations)[:4])
print(df_r2.r.agg(agg_operations)[:4])
print(df_r1.r.agg(agg_operations)==df_r2.r.agg(agg_operations))

mean       0.002731
median    -0.009649
max       12.109637
min       -0.872727
Name: r, dtype: float64
mean       0.003762
median    -0.015353
max       12.109637
min       -0.872727
Name: r, dtype: float64
mean       0.003762
median    -0.015353
max       12.109637
min       -0.872727
Name: r, dtype: float64
mean      True
median    True
max       True
min       True
std       True
size      True
Name: r, dtype: bool


In [84]:
print(df_r.loc[index,'r'].reset_index('code')['r'].resample('Q').agg(agg_operations))
print(df_r1.loc[index,'r'].reset_index('code')['r'].resample('Q').agg(agg_operations))

                mean    median       max       min       std  size
date                                                              
2011-09-30  0.076663  0.076663  0.076663  0.076663       NaN     1
2011-12-31 -0.035069 -0.034591  0.081654 -0.134314  0.072813    13
2012-03-31  0.113472  0.092634  0.436782 -0.046875  0.112979    18
2012-06-30       NaN       NaN       NaN       NaN       NaN     0
2012-09-30       NaN       NaN       NaN       NaN       NaN     0
2012-12-31  0.136266  0.136266  0.146096  0.126437  0.013901     2
2013-03-31       NaN       NaN       NaN       NaN       NaN     0
2013-06-30       NaN       NaN       NaN       NaN       NaN     0
2013-09-30       NaN       NaN       NaN       NaN       NaN     0
2013-12-31       NaN       NaN       NaN       NaN       NaN     0
2014-03-31       NaN       NaN       NaN       NaN       NaN     0
2014-06-30       NaN       NaN       NaN       NaN       NaN     0
2014-09-30       NaN       NaN       NaN       NaN       NaN  

In [85]:
print(df_r.loc[index,'r'].agg(agg_operations))
print(df_r1.loc[index,'r'].agg(agg_operations))

mean         0.064221
median       0.058891
max          1.204987
min         -0.554711
std          0.126345
size      1302.000000
Name: r, dtype: float64
mean         0.073225
median       0.048387
max          1.204987
min         -0.554711
std          0.145928
size      1302.000000
Name: r, dtype: float64


In [122]:
print(df.index.is_lexsorted())
# df.groupby('code').apply(lambda x:np.arange(1,len(x+1))).head()
df['tmp'] = 1
df['idx'] = df.reset_index('code').groupby('code')['tmp'].expanding(1).sum().astype('int')
print(df['idx'].head())
del df['tmp']

KeyError: 'tmp'

In [ ]:
# df_target_not_trash = df[stock_mask & strategy_mask]
print(df_target_not_trash.shape)
for i in range(1,6):
#     df_target_not_trash['idx_diff{}'.format(i)] = df_target_not_trash.groupby('code')['idx'].diff(i)
    # 打印连续超跌i天的样本数量
    print('idx_diff{0}=={0}:'.format(i),(df_target_not_trash['idx_diff{}'.format(i)]==i).sum())
    
# df_target_not_trash = df_target_not_trash.join(df_r,how='inner',rsuffix='r_')
df_target_not_trash = df_target_not_trash.join(df_r2[['r']],how='inner',rsuffix='2')

# 检查拼表后数据是否完整
print(df_target_not_trash.shape)
print('间隔超跌的样本数：',(df_target_not_trash['idx_diff1']>1).sum())
print(df_target_not_trash[df_target_not_trash['idx_diff1']>1]['idx_diff1'].head())
print('第一次超跌的样本数：',df_target_not_trash['idx_diff1'].isnull().sum())
print(df_target_not_trash[df_target_not_trash['idx_diff1'].isnull()]['idx_diff1'].head())

In [128]:
# df_target = df.loc[index]
# print(df.columns)
# df_target = df_target.join(df_r,how='inner',rsuffix='_r')
# df_target = df_target.join(df_r2,how='inner',rsuffix='_r2')
# print(df_target.shape)
for i in range(1,6):
#     df_target['idx_diff{}'.format(i)] = df_target.groupby('code')['idx'].diff(i)
    # 打印连续超跌i天的样本数量
    print('idx_diff{0}=={0}:'.format(i),(df_target['idx_diff{}'.format(i)]==i).sum())
    print(df_target.loc[df_target['idx_diff{}'.format(i)]==i,'r'].agg(agg_operations),'\n')
    print(df_target.loc[df_target['idx_diff{}'.format(i)]==i,'r_r2'].agg(agg_operations),'\n')

for k in [1,5,10,20]:
    print('k={}'.format(k))
    print(df_target.loc[df_target['idx_diff1']>k,'r'].agg(agg_operations),'\n')
    print(df_target.loc[df_target['idx_diff1']>k,'r_r2'].agg(agg_operations),'\n')

print('\nnull')
print(df_target.loc[df_target['idx_diff1'].isnull(),'r'].agg(agg_operations),'\n')
print(df_target.loc[df_target['idx_diff1'].isnull(),'r_r2'].agg(agg_operations),'\n')

print('\nAll')
print(df_target.loc[:,'r'].agg(agg_operations),'\n')
print(df_target.loc[:,'r_r2'].agg(agg_operations),'\n')
    
# df_target = df_target.join(df_r,how='inner',rsuffix='r_')
# 检查拼表后数据是否完整
print(df_target.shape)
print('间隔超跌的样本数：',(df_target['idx_diff1']>1).sum())
print(df_target[df_target['idx_diff1']>1]['idx_diff1'].head())
print('第一次超跌的样本数：',df_target['idx_diff1'].isnull().sum())
print(df_target[df_target['idx_diff1'].isnull()]['idx_diff1'].head())

idx_diff1==1: 592
mean        0.050100
median      0.040513
max         0.655405
min        -0.554711
std         0.112235
size      592.000000
Name: r, dtype: float64 

mean        0.054878
median      0.024129
max         0.655405
min        -0.554711
std         0.132836
size      592.000000
Name: r_r2, dtype: float64 

idx_diff2==2: 323
mean        0.030315
median      0.022945
max         0.551485
min        -0.554711
std         0.096181
size      323.000000
Name: r, dtype: float64 

mean        0.032791
median      0.005263
max         0.551485
min        -0.554711
std         0.121943
size      323.000000
Name: r_r2, dtype: float64 

idx_diff3==3: 194
mean        0.012319
median      0.013333
max         0.253676
min        -0.554711
std         0.093841
size      194.000000
Name: r, dtype: float64 

mean        0.010770
median     -0.002691
max         0.253676
min        -0.554711
std         0.113661
size      194.000000
Name: r_r2, dtype: float64 

idx_diff4==4: 131
mean   

In [129]:
for i in range(1,6):
    print('idx_diff{0}=={0}:'.format(i),(df_target['idx_diff{}'.format(i)]==i).sum())
    print(df_target_not_trash.loc[df_target_not_trash['idx_diff{}'.format(i)]==i,'r'].agg(agg_operations),'\n')
    print(df_target_not_trash.loc[df_target_not_trash['idx_diff{}'.format(i)]==i,'r2'].agg(agg_operations),'\n')
    
for k in [1,5,10,20]:
    print('k={}'.format(k))
    print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1']>k),'r'].agg(agg_operations),'\n')
    print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1']>k),'r2'].agg(agg_operations),'\n')

print('\nNull')
print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1'].isnull()),'r'].agg(agg_operations),'\n')
print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1'].isnull()),'r2'].agg(agg_operations),'\n')

print('\nAll')
print(df_target_not_trash['r'].agg(agg_operations),'\n')
print(df_target_not_trash['r2'].agg(agg_operations),'\n')
# print(df_target_not_trash.loc[df_target_not_trash['idx_diff1']!=1,'r2'].agg(agg_operations),'\n')
# print(df_target_not_trash.loc[(df_target_not_trash['idx_diff1']>5)|(df_target_not_trash['idx_diff1'].isnull()),'r2'].agg(agg_operations),'\n')

df_target_not_trash['1+r'] = df_target_not_trash['r']+1
df_target_not_trash['1+r2'] = df_target_not_trash['r2']+1

df_target_not_trash = df_target_not_trash.sort_values(['date','code'])
print(df_target_not_trash['1+r'][::-1].cumprod(axis=0).tail())
print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0).tail())
print()
print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[500:510])
print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0)[500:510])
print()
print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[200:210])
print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0)[200:210],'\n')

for k in [10,20,50,100,200]:
    print('\nk={}'.format(k))
    print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[k:k+10])
    print(df_target_not_trash['1+r2'][::-1].cumprod(axis=0)[k:k+10])


print(df_target_not_trash['1+r'][::-1].cumprod(axis=0)[:30])
print(df_target_not_trash[['1+r','1+r2','sell_date']][::-1][:30])

print(df_target_not_trash['r'].reset_index('code')['r'].resample('Q').agg(agg_operations))


mean         0.026368
median       0.022989
max          0.655405
min         -0.207692
std          0.069571
size      1728.000000
Name: r, dtype: float64 

mean         0.034785
median       0.015994
max          0.655405
min         -0.492201
std          0.093027
size      1728.000000
Name: r2, dtype: float64 

idx_diff1==1: 592
mean         0.026680
median       0.022405
max          0.268148
min         -0.171271
std          0.061703
size      1064.000000
Name: r, dtype: float64 

mean         0.036855
median       0.015085
max          0.358641
min         -0.492201
std          0.091062
size      1064.000000
Name: r2, dtype: float64 

idx_diff2==2: 323
mean        0.025853
median      0.020886
max         0.242424
min        -0.171271
std         0.061038
size      721.000000
Name: r, dtype: float64 

mean        0.035423
median      0.011073
max         0.358641
min        -0.492201
std         0.091625
size      721.000000
Name: r2, dtype: float64 

idx_diff3==3: 194
mean   

                mean    median       max       min       std  size
date                                                              
2011-12-31 -0.010072 -0.002491  0.081329 -0.103537  0.057601    20
2012-03-31  0.129921  0.123876  0.149573  0.120000  0.012359     5
2012-06-30       NaN       NaN       NaN       NaN       NaN     0
2012-09-30       NaN       NaN       NaN       NaN       NaN     0
2012-12-31  0.049252  0.053061  0.076647  0.019380  0.020764     5
2013-03-31       NaN       NaN       NaN       NaN       NaN     0
2013-06-30 -0.001443 -0.001713  0.094919 -0.057143  0.030376    54
2013-09-30  0.023118  0.026345  0.109537 -0.066289  0.052144    34
2013-12-31  0.053341  0.053341  0.055400  0.051282  0.002912     2
2014-03-31  0.051143  0.062480  0.074101  0.005512  0.031619     4
2014-06-30  0.035240  0.035240  0.078145 -0.007664  0.060676     2
2014-09-30       NaN       NaN       NaN       NaN       NaN     0
2014-12-31       NaN       NaN       NaN       NaN       NaN  

In [22]:
k = 5
ss = df.head(2000)

In [23]:
# ss[features].reset_index('code').groupby('code')[features].rolling(250*k).apply(lambda x:pd.DataFrame(x).rank(pct=True)).tail(10)
# ss[features].groupby('code').rolling(1000).apply(lambda x:pd.Series(x).shape).tail(10)
# ss[features].groupby('code').expanding(1000).apply(lambda x:pd.DataFrame(x).rank(pct=True)).tail(10)
ss[features].groupby('code')[features].rolling(100).apply(lambda x:pd.DataFrame(x).rank(pct=True).loc[len(x)-1,:]).tail(20)

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  after removing the cwd from sys.path.


close/5ma  close/10ma  close/20ma  close/30ma  \
code      code      date                                                        
000001.SZ 000001.SZ 2018-06-12       0.44        0.49        0.39        0.44   
                    2018-06-13       0.33        0.34        0.32        0.37   
                    2018-06-14       0.61        0.59        0.51        0.51   
                    2018-06-15       0.82        0.76        0.67        0.60   
                    2018-06-19       0.34        0.33        0.39        0.35   
                    2018-06-20       0.47        0.43        0.47        0.43   
                    2018-06-21       0.40        0.38        0.46        0.41   
                    2018-06-22       0.49        0.45        0.51        0.47   
                    2018-06-25       0.12        0.15        0.20        0.14   
                    2018-06-26       0.11        0.12        0.18        0.14   
                    2018-06-27       0.07        0.07        0.06        0.05   
                    2018-06-28       0.08        0.05        0.06        0.04   
                    2018-06-29       0.44        0.16        0.12        0.12   
                    2018-07-02       0.08        0.05        0.04        0.03   
                    2018-07-03       0.21        0.10        0.07        0.04   
                    2018-07-04       0.27        0.12        0.08        0.05   
                    2018-07-05       0.41        0.22        0.10        0.07   
                    2018-07-06       0.70        0.33        0.13        0.11   
                    2018-07-09       0.96        0.93        0.50        0.39   
                    2018-07-10       0.93        0.92        0.51        0.40   

                                close/60ma  close/120ma  close/250ma  
code      code      date                                              
000001.SZ 000001.SZ 2018-06-12        0.45         0.07         0.02  
                    2018-06-13        0.33         0.02         0.01  
                    2018-06-14        0.54         0.10         0.04  
                    2018-06-15        0.62         0.16         0.08  
                    2018-06-19        0.33         0.02         0.01  
                    2018-06-20        0.46         0.04         0.02  
                    2018-06-21        0.43         0.04         0.01  
                    2018-06-22        0.45         0.05         0.01  
                    2018-06-25        0.13         0.01         0.01  
                    2018-06-26        0.12         0.01         0.01  
                    2018-06-27        0.06         0.01         0.01  
                    2018-06-28        0.02         0.01         0.01  
                    2018-06-29        0.09         0.03         0.03  
                    2018-07-02        0.01         0.01         0.01  
                    2018-07-03        0.02         0.02         0.02  
                    2018-07-04        0.02         0.02         0.02  
                    2018-07-05        0.03         0.03         0.01  
                    2018-07-06        0.05         0.05         0.04  
                    2018-07-09        0.19         0.09         0.07  
                    2018-07-10        0.19         0.09         0.07

In [33]:
for i in range(5,6):
    w = i*250
    cols = ['p{}_q1%_{}'.format(w,f) for f in features]
    print(cols)
#     print(len(cols))
#     print(len(features))
    df[cols] = df.reset_index('code').groupby('code')[features].rolling(w,min_periods=int(w*0.9)).quantile(0.01)

['p1250_q1%_close/5ma', 'p1250_q1%_close/10ma', 'p1250_q1%_close/20ma', 'p1250_q1%_close/30ma', 'p1250_q1%_close/60ma', 'p1250_q1%_close/120ma', 'p1250_q1%_close/250ma']


In [67]:
mask60 =  (df['close/30ma']<=df['p1250_q1%_close/30ma']*1.03) 
print(mask60.sum(),df['p1250_q1%_close/60ma'].notnull().sum(),mask60.sum()/df['p1250_q1%_close/60ma'].notnull().sum())

df_r.loc[df.index[mask60],'r'].agg(agg_operations)

4891 277147 0.01764767433888875


C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pandas\core\indexing.py:910: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_nested_tuple(tup)


mean         0.017229
median       0.000000
max          0.707305
min         -0.554711
std          0.099349
size      4891.000000
Name: r, dtype: float64

In [76]:
df_d_basic.tail()

close  turnover_rate  turnover_rate_f  volume_ratio  \
code      date                                                              
603997.SH 2020-04-07  12.01         9.5215        39.344398          2.26   
          2020-04-08  10.81         6.7385        27.844700          1.13   
          2020-04-09   9.73         2.4960        10.313700          0.36   
          2020-04-10   9.01         8.2926        34.266300          1.14   
          2020-04-13   9.08         6.4077        26.477800          0.86   

                             pe     pe_ttm      pb      ps  ps_ttm  \
code      date                                                       
603997.SH 2020-04-07  41.295300  41.295300  2.7027  0.6829  0.6829   
          2020-04-08  37.169300  37.169300  2.4326  0.6147  0.6147   
          2020-04-09  33.455799  33.455799  2.1896  0.5533  0.5533   
          2020-04-10  30.980101  30.980101  2.0276  0.5123  0.5123   
          2020-04-13  31.220800  31.220800  2.0433  0.5163  0.5163   

                        total_share   float_share    free_share      total_mv  \
code      date                                                                  
603997.SH 2020-04-07  102360.289062  63235.328125  15303.180664  1.229347e+06   
          2020-04-08  102360.289062  63235.328125  15303.180664  1.106515e+06   
          2020-04-09  102360.289062  63235.328125  15303.180664  9.959656e+05   
          2020-04-10  102360.289062  63235.328125  15303.180664  9.222662e+05   
          2020-04-13  102360.289062  63235.328125  15303.180664  9.294314e+05   

                          circ_mv   pb*pe_ttm       pb*pe  
code      date                                             
603997.SH 2020-04-07  759456.3125  111.608803  111.608803  
          2020-04-08  683573.9375   90.418037   90.418037  
          2020-04-09  615279.7500   73.254814   73.254814  
          2020-04-10  569750.3125   62.815254   62.815254  
          2020-04-13  574176.8125   63.793457   63.793457